**DAND_P5_Final_Project_Report**

Scott Tse
3/30/17

**1. Summarize for us the goal of this project and how machine learning is useful in trying to accomplish it. As part of your answer, give some background on the dataset and how it can be used to answer the project question. Were there any outliers in the data when you got it, and how did you handle those?  [relevant rubric items: “data exploration”, “outlier investigation”]**

The goal of this project is to be able to take a dataset made up of financial information and emails correspoding to various individuals (mostly Enron Sr. Management) and build a program that is able to determine who is a Person of Interest related to the Enron scandal from this information.  

The email dataset, which originally was comprised of around 500K emails is availabe [here](https://www.cs.cmu.edu/~./enron/). It is made up of emails from mostly Sr. Management at Enron and was used during the DOJ investigation of Enron and key executives. In its current form, the emails have been cleaned by individuals at MIT and SRI, and some personal information has been redacted at requested of affected indivuduals.

The Enron scandal is one of the largest corporate scandals ever to take place in the US. At its peak, it was a diversifed energy, commodities and services company, dealing in natural gas, energy trading and even broadband services. By revenue, it was at one point the 7th largest company in the US and a stock market darling througout the late 1990s. In the end, Enron was undone by its fraudulent accounting practices which were revealed to be misleading and largely falsified. The scandal not only caused the bankrupty and destruction of Enron but also that of the veritable accounting firm, Arthur Anderson, which had been in existance for almost 90 years. A detailed history of the scandal an be found on [Wikipedia](https://en.wikipedia.org/wiki/Enron_scandal).

Machine Learning is useful for this goal because it allows us to use a technique called Supervised Classification which can take a labeled dataset with various features and create a model to predict which class each data point belongs to. The Enron dataset made available to us for this project is indeed a labeled dataset because we have the following information:

1.  Who is a person of interest. Since the Enron case was concluded years ago, we know who was indicted and or prosectued by the US Goverment as a result of the corporate scandal. The class of POI a person belongs to is used as the 'label' in this exercise.
2.  We have data to allow several features associated with each individual. The features are either financial features such as salary, bonus, exercised stock options, or features derived from the emails such as number recieived from a POI, emails sent to a POI, and total number of emails.

After doing some exploratory data analysis detailed in [P5_EDA_notebook.ipnb](https://github.com/sky-t/DAND/blob/master/P5_Intro_Machine_Learning/P5_EDA_notebook.ipynb), we can summarize some facts about the dataset:

* There are 146 data points (since each datapoint is a dictionary key with feature attributes as values, this also means there are 146 individuals in the dataset
* Within the 146 individuals, there are 18 persons of interest. Since POI is our target classification variable, this means that this dataset suffers from class imbalance where the percentage of each class is not roughly equal.
* There are 20 features if we exclude 'poi' which is the target
* We have several features with many missing values. After converting the data from python dictionary to pandas dataframe, then converting appropriate data types to numeric, We can see that features such as 'loan_advances', director_fees, and restricted_stock_deferred have very few non null values (<20%).
* Several individuals have many features with null values. 7 individuals have more than 80% of their features with null values.

Regarding outliers, they do exist in the dataset. The outliers I found were the following:

1.  The row of data associated with the individual entry 'TOTAL'. This was an obvious outlier caused by a spreadsheet artifact. It is the sum aggregation for all individuals in the dataset and showed up in several visualizations of the financial features as a datapoint with very high values.
2.  The row of data associated with the individual entry 'THE TRAVEL AGENCY IN THE PARK'. This was not an outlier with extreme values but one with data missing from most features stemming from the fact that it is not a datapoint for an actual individual person, but for a business. It showed up in the dataset since it did have a value in the feature 'payments'. Similarly, I decided to exclude 'LOCKHART EUGENE E' since this inddivual had nulls for all features, rendering the datapoint useless. As noted above, several other individuals have > 80% of their features as null values but I decided to leave them in the model, understanding that their contribution to prediction will likely be quite little.
3.  There were other extreme values for individuals such as Kenneth Lay and Jeff Skilling, some of the top executives in the dataset. Although the values were extreme, I left them in the dataset for modeling since I felt they were valid data points.



**2. What features did you end up using in your POI identifier, and what selection process did you use to pick them? Did you have to do any scaling? Why or why not? As part of the assignment, you should attempt to engineer your own feature that does not come ready-made in the dataset -- explain what feature you tried to make, and the rationale behind it. (You do not necessarily have to use it in the final analysis, only engineer and test it.) In your feature selection step, if you used an algorithm like a decision tree, please also give the feature importances of the features that you use, and if you used an automated feature selection function like SelectKBest, please report the feature scores and reasons for your choice of parameter values.  [relevant rubric items: “create new features”, “properly scale features”, “intelligently select feature”]**

This was my feature engineering process:

My first step was to do some visualizations of the features, comparing numerical data based on POI vs non-POI. I mainly used boxplots for this task. From these boxplots, I could visually see which features showed a qualitative difference between POI and non-POI which helped give me an intiution for which featurs may be important in classification. After this initial EDA, I had some idea of which features to exlude from the dataset even from the first step such as email_address (text variable) and loan_advances (only 3 datapoints).

At this point, I created and added features to the dictionary. I decided to add two new features. The first was `total_salary_ratio` which was created by calculating the ratio of total_payments to salary. My reasoning was that perhaps this ratio as a feature may have predicitve power. For instance, if a person had a very high ratio of total payments to base salary, perhaps they may be liklier to be a POI. Another feature I created was `payments_plus_stock`. My reasoning for this feature was to test the idea that perhaps an individual's total payout from stock and payments may have predicitive power. Interestingly, as explained later  in this report, the feature set I chose based on SelectKBest actually did include both new features I created.  

Then, starting with the full set of features minus the obvious ones for removal described earlier, I used a pipeline combined with GridSeachCV approach. Within the pipeline itself, I used SelectKBest to reduce the feature set to 12 (returned as optimum by GridSeachCV), as compiled in the following list, where 'poi' is the target variable (feature score and P-values included): 

|Feature                |Score        |P-value   |
|-----------------------|-------------|----------|
|exercised_stock_options|24.82        |0.000     |
|bonus                  |20.79        |0.000     |
|salary                 |18.29        |0.000     |
|payments_plus_stock    |17.81        |0.000     |
|deferred_income        |11.46        |0.001     |
|long_term_incentive    |9.92         |0.002     |
|restricted_stock       |9.21         |0.003     |
|total_payments         |8.77         |0.004     |
|shared_receipt_with_poi|8.59         |0.004     |
|from_poi_to_this_person|5.24         |0.024     |
|other                  |4.19         |0.043     |
|from_this_person_to_poi|2.38         |0.125     |
|director_fees          |2.13         |0.147     |
|to_messages            |1.65         |0.147     |
|restricted_stock_deferred|0.07        |0.636     |
|director_fees          |0.07         |0.798     |
|total_salary_ratio     |0.05         |0.819    |


Within the pipeline, I then used `preprocessing.StandardScaler()` to scale the features, then applied PCA via `sklearn.decomposition.PCA()`. For PCA, the optimum number of components was 4.




    

**3. What algorithm did you end up using? What other one(s) did you try? How did model performance differ between algorithms?  [relevant rubric item: “pick an algorithm”]**

I chose to use the Gaussian Naive Bayes algorithm from sklearn.naive_bayes.GaussianNB(). I chose this because of the approaches I tried, it gave the best combination of Accuracy, Precision, Recall and F1. After playing with other algorithms such as Decision Tree and Random Forest, I still found the Naive Bayes to have the best metrics when run through the tester.py testing functions. I was able to get Accuracy scores for the other algorithms quite similar to Naive Bayes (~.84 - .86) but the Precision and Recall were not able to match.  Here is a comparison of how some of the the various classifiers permutations fared in my initial analysis:

|Classifier      |Parameters (non default)|Features                                  |Accurracy|Precision|Recall |
|----------------|-----------------------|-------------------------------------------|---------|---------|-------|
|GaussianNB      |None                   |All+total_salary_ratio,payments_plus_stock |0.37680  |0.15912  |0.85750|
|DecisionTree    |min_samples_split=2|Same as above|0.79067|0.23708|0.25700|
|RandomForest    |n_estimators=5         |Same as above|0.84127	|0.33842|0.19950|	
|GaussianNB|None|bonus,salary,payments_plus_stock,shared_receipt_with_poi,total_stock_value, exercised_stock_options, total_payments,deferred_income,restricted_stock,long_term_incentive|0.84380  |0.40058	 |0.34550|
|DecisionTree    |min_samples_split=2|Same as above|0.80793		|0.28030		|0.28100	|
|RandomForest    |n_estimators=5         |Same as above| 0.84573		|0.37379	|0.23250	|	
|RandomForest    |n_estimators=10         |Same as above|0.85787|0.38077	|0.15800	|	
|RandomForest    |n_estimators=15         |Same as above|0.85273	|0.39148		|0.18850		|
|**GassianNB (in pipeline)**|None|12 listed in table above, with PCA applied to reduce to 4 components|0.84893|0.42231	|0.36150	|


**4. What does it mean to tune the parameters of an algorithm, and what can happen if you don’t do this well?  How did you tune the parameters of your particular algorithm? (Some algorithms do not have parameters that you need to tune -- if this is the case for the one you picked, identify and briefly explain how you would have done it for the model that was not your final choice or a different model that does utilize parameter tuning, e.g. a decision tree classifier).  [relevant rubric item: “tune the algorithm”]**

Many of the ML classification algorithms have various parameters that have one or more optional settings. For example, even though I ended up using the Naive Bayes algorithm which is quite simple and does not have parameters that can be tuned, I did play with the Random Forest classifier which has many parameters such as "criterion", "n_estimators" and others summarized in the [documentation](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html). Some of the parameters I manually tried are summarized in the table above. Another way to approach parameter selection is to use GridSearchCV to test every combination of algorithm parameter settings that you choose, and use the algorithm to select the best one according an average of metrics returned with cross validation. For the the final ML approach I selected for this project, I used GridSearchCV to help speed up and automate feature selection and PCA implementation. Because I ended up using GuassianNB (which does not have parameter options to select from) as the classifier within the pipeline object in GridSearchCV, I was not able test multiple classifier parameters. This is a table of the top 10 models returned by GridSearchCV, based on the options I input:

In [20]:
from IPython.display import display, HTML
display(HTML(filename='GSCV_.html'))

,mean_test_score,mean_train_score,param_PCA__n_components,param_PCA__whiten,param_kbest__k,params
rank_test_score,,,,,,
1,0.797406,0.812257,4,True,12,"{u'PCA__n_components': 4, u'kbest__k': 12, u'P..."
2,0.776758,0.782152,5,True,12,"{u'PCA__n_components': 5, u'kbest__k': 12, u'P..."
3,0.772774,0.793918,6,True,10,"{u'PCA__n_components': 6, u'kbest__k': 10, u'P..."
4,0.769393,0.800201,2,True,12,"{u'PCA__n_components': 2, u'kbest__k': 12, u'P..."
5,0.766094,0.788511,4,True,10,"{u'PCA__n_components': 4, u'kbest__k': 10, u'P..."
6,0.762599,0.828608,3,True,20,"{u'PCA__n_components': 3, u'kbest__k': 20, u'P..."
7,0.762126,0.783814,6,True,12,"{u'PCA__n_components': 6, u'kbest__k': 12, u'P..."
8,0.754753,0.821265,3,True,12,"{u'PCA__n_components': 3, u'kbest__k': 12, u'P..."
9,0.754045,0.788882,2,True,14,"{u'PCA__n_components': 2, u'kbest__k': 14, u'P..."


NOTE: I did happen to try using a DecisionTree classifier with GridSeachCV and this approach is summarized in that latter portion of P5_notebook.ipynb. In that exercise, I chose to explore various levels of two parameters:
* "DT_clf__min_samples_split":[2,3,4]
* "DT_clf__max_depth":[6,4]

**5. What is validation, and what’s a classic mistake you can make if you do it wrong? How did you validate your analysis?  [relevant rubric item: “validation strategy”]**

Validation in the context of machine learning is basically testing how well the model that one has generated on the dataset can work on data outside of that training data -- that is, how generalizable is it to the "real world". A classic mistake if validation is not performed correctly is overfitting, where the model fits the data it was training on very well, but performed poorly when applied to other, often "real world" data. 

In my case, I utilized cross validation tools in sklearn to train my model and select best parameters, specifically GridSearchCV, and then used the provided cross validation approaches in tester.py to acquire the performance metrics. Because the dataset was small, I chose not to split the dataset into test and training before inserting it into my pipeline. Since GridSearchCV uses cross validation approach using StratifiedKFold to basically take random slices of the data to do its selection based on a average of scores, I reasoned that for such a small and imbalanced dataset, it would be better to not seperate a test set before the pipeline.

In addition, the `test_classifier` function in tester.py utilizes StratifiedShuffleSplit in generating test sets for cross validation. Similar to StratifiedKFold, it does this through generating ramdom samples of a subset of the entire dataset for testing, attempting to keep relative class frequencies intact for each fold. This helps guard against being "unlucky" in a single train-test split if most datapoints of one class get shunted to the test or training set, distorting the results.

**6. Give at least 2 evaluation metrics and your average performance for each of them.  Explain an interpretation of your metrics that says something human-understandable about your algorithm’s performance. [relevant rubric item: “usage of evaluation metrics”]**

The evaluation metrics I mostly focused on were Accuracy, Precision, and Recall. Model Accuracy for classification is what percentage of classes the model gets correct during prediction. This is important, but may miss some nuances of model performance. For instance, in a dataset where most people are not POIs, say 18/144, the algorithm may just classify everyone a non POI and still have high accuracy. Using Recall (true positives/(true positives+false negatives) and Precision (True Positive / (True Positive + False Positive) give us a better understanding of how the classifier performs. In my case, I was able to get to these metrics from tester.py:

    Accuracy: 0.84893	Precision: 0.42231	Recall: 0.36150	F1: 0.38955	F2: 0.37222
	Total predictions: 15000	True positives:  723	False positives:  989	False negatives: 1277	
    True negatives: 12011
    
We see here that even though the classifier has decent Accuracy, the Precision and Recall metrics are still quite low. This classifier still has low capability to find true POIs (of all POIs, it only correctly classifies around 36% of them) and a slightly higher but still low capability to not label a person as a POI who is not (out of all people labeled a POI, who many are truly a POI). More work could be done to increase all 3 of these core metrics through feature creation/selection and parameter tuning, as well as exploring other ML algorithms.
